# Topic Modelling

The ultimate goal of topic modeling is to find various topics that are present in COVID-19 publications.

Latent Dirichlet Allocation (LDA) is one of many topic modeling techniques. It was specifically designed for text data.

To use a topic modeling technique, we need (1) a document-term matrix and (2) the number of topics you would like the algorithm to pick up.

Once the topic modeling technique is applied, we need to interpret the results and see if the mix of words in each topic make sense. If they don't make sense, we will try changing up the number of topics, the terms in the document-term matrix, model parameters, or even try a different model.

## Data Cleaning 

In [21]:
import pandas as pd
import pickle

dr = pd.read_pickle('data/covid19_reference_count.pickle')
abstract = pd.read_pickle('data/covid19_abstract.pickle')
abstract30 = abstract[dr>=30]
abstract30_text = abstract30.astype('str')

#clean data first
#Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\<.*?\>', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)
clean_data1 = abstract30_text.apply(round1)

# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

# Let's take a look at the updated text
clean_data2 = clean_data1.apply(round2)


## Let's create our document-term matrix from abstracts

In [66]:
#create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords;

#new_stopwords =['pandemic','study','nan','patient','disease','coronavirus',
#                'health','review','impact','research','may','results',
#               'public']
new_stopwords = ['']
new_stopwords += stopwords.words('english')
cv = CountVectorizer(stop_words=new_stopwords)
data_cv = cv.fit_transform(clean_data2)

# convert to a numerical array
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = clean_data2.index

In [67]:
# Import the necessary modules for LDA with gensim
from gensim import matutils, models
import scipy.sparse
# One of the required inputs is a term-document matrix
tdm = data_dtm.transpose()
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [68]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term), we need to specify two other parameters - the number of topics and the number of passes. Let's start the number of topics at 2, see if the results make sense, and increase the number from there.

In [69]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.007*"pandemic" + 0.007*"health" + 0.006*"coronavirus" + 0.005*"virus" + 0.005*"protein" + 0.004*"study" + 0.004*"viral" + 0.004*"data" + 0.004*"disease" + 0.003*"human"'),
 (1,
  '0.036*"nan" + 0.013*"patients" + 0.008*"infection" + 0.008*"cells" + 0.007*"disease" + 0.006*"respiratory" + 0.005*"severe" + 0.005*"viral" + 0.005*"expression" + 0.004*"coronavirus"')]

In [70]:
# LDA for num_topics = 3
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

[(0,
  '0.011*"health" + 0.008*"pandemic" + 0.005*"study" + 0.005*"research" + 0.005*"data" + 0.005*"social" + 0.004*"public" + 0.004*"results" + 0.003*"care" + 0.003*"disease"'),
 (1,
  '0.009*"viral" + 0.008*"protein" + 0.008*"virus" + 0.007*"coronavirus" + 0.007*"human" + 0.006*"cells" + 0.006*"infection" + 0.005*"pandemic" + 0.005*"spike" + 0.005*"disease"'),
 (2,
  '0.075*"nan" + 0.020*"patients" + 0.008*"disease" + 0.007*"de" + 0.006*"clinical" + 0.006*"severe" + 0.006*"infection" + 0.006*"risk" + 0.005*"respiratory" + 0.005*"coronavirus"')]

In [71]:
# LDA for num_topics = 4
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda.print_topics()

[(0,
  '0.011*"health" + 0.008*"pandemic" + 0.005*"study" + 0.005*"research" + 0.005*"data" + 0.004*"social" + 0.004*"care" + 0.004*"patients" + 0.004*"results" + 0.004*"public"'),
 (1,
  '0.012*"coronavirus" + 0.008*"respiratory" + 0.007*"disease" + 0.007*"virus" + 0.007*"de" + 0.006*"patients" + 0.005*"severe" + 0.005*"pandemic" + 0.005*"syndrome" + 0.005*"la"'),
 (2,
  '0.011*"protein" + 0.009*"virus" + 0.009*"viral" + 0.007*"coronavirus" + 0.007*"human" + 0.006*"spike" + 0.005*"pandemic" + 0.005*"host" + 0.005*"binding" + 0.005*"infection"'),
 (3,
  '0.071*"nan" + 0.013*"cells" + 0.009*"patients" + 0.009*"infection" + 0.007*"expression" + 0.007*"disease" + 0.006*"lung" + 0.006*"respiratory" + 0.005*"de" + 0.005*"severe"')]

Comments: These topics aren't looking too great. I tried to add some stop words. 
Let's try modifying our terms list as well.

## Try topic modelling using Nouns only from the abstracts

One popular trick is to look only at terms that are from one part of speech (only nouns, only adjectives, etc.).

In [53]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

# Apply the nouns function to the abstracts to filter only on nouns
data_nouns = pd.Series(clean_data2.apply(nouns))

In [72]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

#new_stopwords =['pandemic','study','nan','patient','disease','coronavirus',
#                'health','review','impact','research','may','results',
#               'public']
new_stopwords=['da','la','em','se','que','por','nan']
new_stopwords += stopwords.words('english')
# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=new_stopwords)
data_cvn = cvn.fit_transform(data_nouns)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
data_dtmn.index = data_nouns.index

In [73]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [74]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.013*"protein" + 0.012*"virus" + 0.012*"coronavirus" + 0.011*"infection" + 0.011*"cells" + 0.008*"disease" + 0.008*"cell" + 0.007*"respiratory" + 0.007*"host" + 0.007*"spike"'),
 (1,
  '0.017*"health" + 0.011*"patients" + 0.008*"data" + 0.008*"disease" + 0.008*"study" + 0.008*"research" + 0.006*"coronavirus" + 0.006*"results" + 0.006*"pandemic" + 0.006*"care"')]

In [75]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.014*"protein" + 0.014*"virus" + 0.013*"infection" + 0.012*"coronavirus" + 0.012*"cells" + 0.010*"disease" + 0.009*"cell" + 0.008*"respiratory" + 0.008*"host" + 0.007*"spike"'),
 (1,
  '0.009*"drugs" + 0.007*"drug" + 0.007*"research" + 0.006*"coronavirus" + 0.006*"inhibitors" + 0.005*"compounds" + 0.005*"protease" + 0.004*"analysis" + 0.004*"results" + 0.004*"studies"'),
 (2,
  '0.020*"health" + 0.012*"patients" + 0.008*"study" + 0.008*"disease" + 0.008*"data" + 0.007*"care" + 0.007*"research" + 0.006*"pandemic" + 0.006*"coronavirus" + 0.006*"results"')]

In [76]:
# Let's try topics = 4
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.015*"protein" + 0.014*"virus" + 0.013*"cells" + 0.013*"coronavirus" + 0.012*"infection" + 0.009*"cell" + 0.009*"disease" + 0.008*"host" + 0.008*"spike" + 0.007*"respiratory"'),
 (1,
  '0.029*"health" + 0.013*"research" + 0.010*"study" + 0.008*"data" + 0.006*"care" + 0.006*"information" + 0.006*"pandemic" + 0.005*"results" + 0.005*"media" + 0.005*"management"'),
 (2,
  '0.024*"patients" + 0.014*"disease" + 0.010*"coronavirus" + 0.010*"cases" + 0.008*"risk" + 0.008*"data" + 0.008*"infection" + 0.008*"health" + 0.007*"treatment" + 0.007*"results"'),
 (3,
  '0.005*"research" + 0.005*"study" + 0.005*"generation" + 0.004*"authors" + 0.004*"results" + 0.004*"consumption" + 0.004*"data" + 0.004*"pandemic" + 0.004*"information" + 0.004*"estate"')]

## Try topic modelling using Nouns + Adjectives from the abstracts

One popular trick is to look only at terms that are from one part of speech (only nouns, only adjectives, etc.).

In [ ]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(data_clean.transcript.apply(nouns_adj))
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.transcript)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names())
data_dtmna.index = data_nouns_adj.index

In [ ]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [ ]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

In [ ]:
# Let's start with 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

In [ ]:
# Let's start with 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

further work to be done:
1. more data cleaning, e.g., cells and cell should be one word.
2. try to add more stop words